In [16]:
from FusionNet import * 
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# hyperparameters
batch_size = 1
img_size = 256
lr = 0.0002
epoch = 100


In [18]:
import shutil, os

train_img_dir = "train/"
dir = os.path.join(train_img_dir + '.ipynb_checkpoints')
if os.path.exists(dir) and os.path.isdir(dir):
        shutil.rmtree(dir)
train_img_data = dset.ImageFolder(root=train_img_dir, transform = transforms.Compose([
                                            transforms.ToTensor(),
                                            ])) #edited
train_img_batch = data.DataLoader(train_img_data, batch_size=batch_size,
                            shuffle=False , num_workers=2) #edited

val_img_dir = "val/"
dir = os.path.join(val_img_dir + '.ipynb_checkpoints')
if os.path.exists(dir) and os.path.isdir(dir):
        shutil.rmtree(dir)
val_img_data = dset.ImageFolder(root=val_img_dir, transform = transforms.Compose([
                                            transforms.ToTensor(),
                                            ])) #edited
val_img_batch = data.DataLoader(val_img_data, batch_size=batch_size,
                            shuffle=False, num_workers=2)



In [19]:
generator = nn.DataParallel(FusionGenerator(3,3,64),device_ids=[0]).cuda()


------Initiating FusionNet------


In [20]:
#torch.nn.Module.dump_patches = True
try:
    generator = torch.load('./model/model_fusionnet.pkl')
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass

C:\Users\jackw\AppData\Local\Temp\ipykernel_38412\4244458258.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator = torch.load('./model/model_fusionnet.pkl')



--------model restored--------


In [21]:
# loss function & optimizer
loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(generator.parameters(),lr=lr)


In [22]:
# training

try:
    file = open('./fusionnet_mse_fu_loss', 'r')
    i = len(file.readlines()) + 1
    file.close()
except:
    i = 1

file = open('./fusionnet_mse_fu_loss', 'a')

while i != epoch:
    for _,(image,label) in enumerate(train_img_batch):
        satel_image, label_image = torch.chunk(image, chunks=2, dim=3) #image.size() = [1,3,256,512], 1=batch size, 3 = number of channel
        
        gen_optimizer.zero_grad()

        x = satel_image.cuda() #edited
        y_ = label_image.cuda() #edited
        y = generator.forward(x)
        
        loss = loss_func(y,y_)
        loss.backward()
        gen_optimizer.step()               
        
        if _ % 100 == 0 :
            print(i,_)
            print(loss)
            file.write(str(loss)+"\n")
            v_utils.save_image(y.cpu().data,"./result_train/gen_image_{}_{}.png".format(i,_))
            torch.save(generator,'./model/model_fusionnet.pkl')
    i+=1

file.close()

49 0
tensor(0.1532, device='cuda:0', grad_fn=<MseLossBackward0>)
49 100
tensor(0.0341, device='cuda:0', grad_fn=<MseLossBackward0>)
49 200
tensor(0.0315, device='cuda:0', grad_fn=<MseLossBackward0>)
49 300
tensor(0.0540, device='cuda:0', grad_fn=<MseLossBackward0>)
49 400
tensor(0.0516, device='cuda:0', grad_fn=<MseLossBackward0>)
49 500
tensor(0.0478, device='cuda:0', grad_fn=<MseLossBackward0>)
49 600
tensor(0.0479, device='cuda:0', grad_fn=<MseLossBackward0>)
49 700
tensor(0.0483, device='cuda:0', grad_fn=<MseLossBackward0>)
49 800
tensor(0.0527, device='cuda:0', grad_fn=<MseLossBackward0>)
49 900
tensor(0.0543, device='cuda:0', grad_fn=<MseLossBackward0>)
49 1000
tensor(0.0346, device='cuda:0', grad_fn=<MseLossBackward0>)
49 1100
tensor(0.0317, device='cuda:0', grad_fn=<MseLossBackward0>)
49 1200
tensor(0.0390, device='cuda:0', grad_fn=<MseLossBackward0>)
49 1300
tensor(0.0488, device='cuda:0', grad_fn=<MseLossBackward0>)
49 1400
tensor(0.0434, device='cuda:0', grad_fn=<MseLossBack

KeyboardInterrupt: 

In [ ]:
file.write(str(loss)+"\n")

In [ ]:
# validate
generator = torch.load('./model/model_fusionnet.pkl')

#,map_location=lambda storage, loc: storage.cuda())


for _,(image,label) in enumerate(val_img_batch):
    satel_image, label_image = torch.chunk(image, chunks=2, dim=3) 
    print(_)
    x = satel_image
    y_ = label_image
    y = generator.forward(x)

    v_utils.save_image(y.cpu().data,"./result_val/gen_image_{}.png".format(_))

In [ ]:
val_img_data